In [1]:
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install duckduckgo-search==5.3.0 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import json
import os
import re
from datetime import datetime

import requests
from crewai import Agent, Crew, Process, Task
from google.colab import userdata
from IPython.display import Markdown
from langchain.agents import load_tools
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI


def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]


os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Data Source

In [3]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [15]:
response = search_tool.run("Bitcoin")
format_response(response)

["snippet: The cryptocurrency's creator—who went by Satoshi Nakamoto, but whose real identity remains unknown—invented bitcoin the following year, and dreamed of creating an international currency that would operate outside the control of governments or central banks., title: What's the Deal With the Bitcoin Halving?, link: https://www.msn.com/en-us/money/markets/the-countdown-to-bitcoins-halving-is-on-heres-what-to-know/ar-BB1lMW8j, date: 2024-04-17T15:07:18+00:00, source: Time on MSN.com",
 'snippet: In a recent discussion, experts highlighted bitcoin miners as a unique investment opportunities in the evolving crypto and energy sectors., title: Bitcoin mining, market experts discuss growing role in energy grid, link: https://www.thestreet.com/crypto/innovation/bitcoin-mining-market-experts-discuss-growing-role-in-energy-grid, date: 2024-04-17T17:04:00+00:00, source: TheStreet.com',
 'snippet: As bitcoin hurtles toward its next supply cut—known as a halving—Goldman Sachs\ufeff analyst

In [16]:
response = search_tool.run("Ethereum")
format_response(response)

['snippet: Exec claims Ethereum L2s can "steal user funds." Industry figures propose multi-signatories to eliminate single points of failure. In a surprising disclosure, it has been revealed that the safety of user funds in Ethereum [ETH] L2s is not guaranteed., title: \'Ethereum L2s can steal user funds right now\' - Are the allegations true?, link: https://www.msn.com/en-us/money/technology/ethereum-l2s-can-steal-user-funds-right-now-are-the-allegations-true/ar-BB1lNgrW, date: 2024-04-17T16:00:07+00:00, source: AMBCrypto on MSN.com',
 "snippet: A major Ethereum (CRYPTO: ETH) investor reportedly lost over $4.5 million in two failed attempts to leverage the cryptocurrency. The investor then sold $33 million worth of ETH to settle the debt. What Happened: According to the Lookonchain report,, title: Crypto Whale's Ethereum Long Strategy Goes Awry, Resulting In $4.5M Loss: Report, link: https://www.msn.com/en-us/money/savingandinvesting/crypto-whale-s-ethereum-long-strategy-goes-awry-res

In [17]:
response = search_tool.run("Cryptocurrency")
format_response(response)

["snippet: The cryptocurrency community has been electrified by the meteoric rise of Option2Trade (O2T), which has seen its value increase by an astounding 500% following its recognition as the 'Dogwifhat Killer., title: This Cryptocurrency Value Skyrockets 500% Following its Recognition As the 'Dogwifhat Killer', link: https://www.msn.com/en-us/money/markets/this-cryptocurrency-value-skyrockets-500-following-its-recognition-as-the-dogwifhat-killer/ar-BB1lNkkq, date: 2024-04-17T15:48:50+00:00, source: Cryptopolitan on MSN.com",
 'snippet: Stay updated on recently launched cryptocurrencies, new listings, and presales. Discover potentially rewarding opportunities that interest investors., title: New Cryptocurrency Releases, Listings, & Presales Today — Web3 TON Token, BlackCardCoin, BonkCon, link: https://insidebitcoins.com/news/new-cryptocurrency-releases-listings-presales-today-web3-ton-token-blackcardcoin-bonkcon, date: 2024-04-17T16:43:00+00:00, source: Inside Bitcoins',
 'snippet: I

## Tools

In [4]:
human_tools = load_tools(["human"])

In [5]:
llm = ChatOpenAI(
    model="gpt-4-turbo-2024-04-09"
    # model="gpt-3.5-turbo"
)

## Agents

In [6]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrencies the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)

analyst = Agent(
    role="Cryptocurrency Analyst",
    goal="""Create a short summary of the market news. Specify the current trend - up, down or sideways.
    For each requested cryptocurrency asset, specify a number between 0 and 100, where 0 is
    extreme fear and 100 is extreme greed.""",
    backstory="""You're widely accepted as the best cryptocurrency analyst that
    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding
    of human psychology. You understand macro factors and combine those multiple
    theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.

    You understand news, their titles and information, but you look at those with a
    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization
    that explains very complex market topics in simple to understand terms.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
)

writer = Agent(
    role="Cryptocurrency Newsletter Writer",
    goal="""Write an insightful, compelling and informative 3 paragraph
    long newsletter based on the cryptocurrency report.""",
    backstory="""You're deeply familiar with all cryptocurrency assets and blockchain technology. You understand
    complex concepts and create compelling stories and narratives that resonate with wider audiences.

    You're a master of communicating effectively and create narratives that engage the audience.
    Your writing focuses on conciseness, clearness and easy to read (flow) formatting.

    Some of your writing techniques include:

    - Start with a 3 bullet list (executive summary) of the most importannt points
    - Use bullet points and subheadings to make text easier to understand and read
    - Use analogies to convey complex ideas
    - Write compelling introductions and conclusions that make your work memorable

    You writing transforms even dry and most technical texts into
    a pleasant and interesting read.""",
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=True,
)

## Tasks

In [7]:
get_cryptocurrencies_list = Task(
    description=f"ask which cryptocurrencies the customer is interested in.",
    expected_output="""Create a list of the cryptocurrencies that the human wants you to research.\n\n
    eg. cryptocurrencies = 'BTC, ETH' """,
    agent=customer_communicator,
)

get_news = Task(
    description=f"""Take the list of cryptocurrencies and always include BTC and ETH to it (if not requested).
    Use the search tool to search for each asset individually.

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""A summary of the overall cryptocurrency market and one sentence summary for
    each customer requested asset. Include a fear/greed score for each asset based on the news. Use the format:

    <CRYPTO_ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent=analyst,
    context=[get_cryptocurrencies_list],
)

write_newsletter = Task(
    description=f"""
    Use the cryptocurrency report to create a newsletter that is brief and highlights the most important points.
    Focus on the news, trend and fear/greed score. What are the near and future considerations?
    The newsletter is written daily and should be accessible to a wide audience.
    Don't rely on hype, but be clear and factual.""",
    expected_output="""An eloquent 3 paragraphs newsletter formated as markdown
    in an easy readable manner. It should contain:

    - 3 bullets executive summary
    - introduction - set up the overall picture and spike up the interest (hook)
    - main part - provides the meat of the analysis including news summary and fear/greed scores
    - summary - key facts and a concrete future trend prediction - up, down or sideways
    """,
    agent=writer,
    context=[get_news],
)

## Crew

In [8]:
crew = Crew(
    agents=[customer_communicator, analyst, writer],
    tasks=[get_cryptocurrencies_list, get_news, write_newsletter],
    verbose=2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

In [9]:
results = crew.kickoff()

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: ask which cryptocurrencies the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: To determine which cryptocurrencies the customer is interested in, I need to gather information directly from the customer. Since I have a team member specialized in communication with customers, I will delegate this task to the Senior cryptocurrency customer communicator.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior cryptocurrency customer communicator", "task": "Inquire about customer's interest in cryptocurrencies", "context": "We need to find out which specific cryptocurrencies the customer is currently interested in or considering for investment, so we can provide tailored information and support. Please reach out to the customer through our standard communication channels and ask them to list the cryptocurrencies they are interested in."}


> Entering new CrewAgentExecutor chain...
Th

In [11]:
list(results.keys())

['final_output', 'tasks_outputs']

In [13]:
len(results["tasks_outputs"])

3

In [14]:
Markdown(results["tasks_outputs"][1].exported_output)

**Overall Cryptocurrency Market Summary:**
The cryptocurrency market is currently navigating through a phase of recovery from recent shocks, with an overlay of vulnerability to external geopolitical and economic factors. The sentiment in the market is cautiously optimistic but shows readiness to react swiftly to any adverse global events. Investors are encouraged to stay updated and consider the implications of market volatility and potential external disruptions on cryptocurrency values.

**Bitcoin (BTC)**
News Summary: Bitcoin saw a minor profit gain of 2.33%, pushing its value to $63,815, though it experienced a downturn due to broader market factors including geopolitical tensions and a flash crash.
Trend Prediction: The trend is sideways to slightly bullish, with an emphasis on recovery from the recent crash, though vulnerable to external shocks.
Fear/Greed Score: 45 (Moderate fear due to recent volatility and external uncertainties).

**Ethereum (ETH)**
News Summary: Ethereum mirrored Bitcoin's movements with minor gains but faced impacts from market-wide sell-offs and external concerns.
Trend Prediction: The trend remains sideways, indicating ongoing recovery efforts post-crash amid a cautious market atmosphere.
Fear/Greed Score: 50 (Neutral, balancing recent gains against prevailing market fears).

**Solana (SOL)**
News Summary: Affected by the general market downturn, Solana has been a topic of positive discussion within its ecosystem, showing promising activity despite broader market issues.
Trend Prediction: Slightly bullish within its ecosystem but generally sideways in the broader market.
Fear/Greed Score: 55 (Slight greed, indicating a relatively better position within its ecosystem compared to the broader market).

**Dogecoin (DOGE)**
News Summary: Dogecoin experienced significant volatility with a recent steep drop of 20%, but there is optimism about a potential rebound driven by community events like Doge Day.
Trend Prediction: Currently bearish but with potential for a bullish reversal if upcoming community events can spark renewed interest.
Fear/Greed Score: 30 (Significant fear due to recent sharp declines, yet there is speculative potential for a rebound).

In [10]:
Markdown(results["final_output"])

**Cryptocurrency Market Daily Digest**

**Executive Summary:**
- **Bitcoin:** Minor profit gains with a fear/greed score of 45, indicating caution due to external shocks. 
- **Ethereum:** Shows slight improvements, maintaining a cautious stance with a fear/greed score of 50.
- **Solana and Dogecoin:** Solana is modestly bullish within its community, scoring 55, while Dogecoin faces a sharp decline but has potential for recovery, marked by a score of 30.

**Introduction:**
Welcome to today's edition of your trusted cryptocurrency newsletter, where we unravel the complexities of the market to bring you the most pertinent updates. As the digital currency landscape continues to evolve, understanding the subtle shifts can help you navigate the potential highs and lows. Today, we delve into the current state of major cryptocurrencies—Bitcoin, Ethereum, Solana, and Dogecoin—each experiencing unique market movements influenced by both internal dynamics and external pressures.

**Detailed Analysis:**
- **Bitcoin:** Despite facing minor profit gains, Bitcoin remains under the influence of external economic shocks, reflecting a fear/greed index of 45. Investors show cautious optimism, balancing between potential growth and prevailing market uncertainties.
- **Ethereum:** Ethereum mirrors a similar pattern to Bitcoin but with a slightly more stable outlook. With a fear/greed score of 50, it indicates a market that is neither too fearful nor overly greedy, suggesting a balanced investor sentiment amid minor gains.
- **Solana:** Standing slightly apart from its counterparts, Solana shows a modest bullish trend within its ecosystem. The fear/greed score of 55 hints at a cautiously optimistic community, possibly driven by specific developments or innovations within the Solana network.
- **Dogecoin:** The meme coin has experienced a significant drop, yet it holds a rebound potential. With the lowest fear/greed score of 30, the sentiment is notably fearful, yet it's this fear that may set the stage for aggressive buying opportunities for risk-tolerant investors.

**Conclusion and Future Outlook:**
As the market treads through a recovery phase, it remains sensitive to geopolitical and economic developments. The mixed signals represented by the fear/greed scores across different cryptocurrencies suggest a market at a crossroads. For Bitcoin and Ethereum, stability might be the theme in the coming days, while Solana could continue to build on its modest gains. Dogecoin, facing the steepest challenges, also presents a high-risk, high-reward scenario for those betting on its recovery. Stay tuned, and navigate wisely, as the only constant in the cryptocurrency market is change.

This newsletter provides a snapshot of today's cryptocurrency climate, ensuring our readers are well-informed and prepared for the market's dynamic nature.